In [1]:
!pip install -q unsloth datasets transformers trl psutil

In [2]:
import torch
import psutil
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-26 14:09:25.933463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766758165.956531     603 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766758165.964357     603 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766758165.985324     603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766758165.985342     603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766758165.985344     603 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 1024
dtype = None
load_in_4bit = True

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Llama-3.2-3B-Instruct',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
dataset = load_dataset("alikhademi98/persian-thinking-scratchpad-v1", split = "train")

README.md:   0%|          | 0.00/402 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [7]:
dataset[:2]

{'instruction': ['۳ به علاوه ۵ چند می\u200cشود؟',
  '۳ به علاوه ۵ چند می\u200cشود؟'],
 'scratchpad_1': ['احتمال می\u200cدهم پاسخ ۸ می\u200cشود باشد، هرچند هنوز قانع نشده\u200cام.',
  'یک پاسخ سرانگشتی ۸ می\u200cشود است، اما ممکن است غلط باشد.'],
 'scratchpad_2': ['پس از بررسی مرحله\u200cبه\u200cمرحله، جواب ۸ می\u200cشود تأیید می\u200cشود.',
  'با بررسی دقیق\u200cتر مشخص می\u200cشود پاسخ درست ۸ می\u200cشود است.'],
 'final_answer': ['۸ می\u200cشود', '۸ می\u200cشود']}

In [8]:
r1_prompt = """تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شونده، تفکر جریان سیال ذهن را تقلید می‌کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
{}
</مسئله>

{}
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    scratchpad_1_list = examples["scratchpad_1"]
    scratchpad_2_list = examples["scratchpad_2"]
    final_answers = examples["final_answer"]

    texts = []
    for instr, sp1, sp2, ans in zip(instructions, scratchpad_1_list, scratchpad_2_list, final_answers):
        text = r1_prompt.format(instr, sp1, sp2) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

In [9]:
dataset = dataset.map(formatting_prompts_func, batched=True, num_proc=2)

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
print(dataset[0]["text"])

تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شونده، تفکر جریان سیال ذهن را تقلید می‌کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
۳ به علاوه ۵ چند می‌شود؟
</مسئله>

احتمال می‌دهم پاسخ ۸ می‌شود باشد، هرچند هنوز قانع نشده‌ام.
پس از بررسی مرحله‌به‌مرحله، جواب ۸ می‌شود تأیید می‌شود.
<|eot_id|>


In [12]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 3 | Total steps = 1,875
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.843200
2,3.942100
3,3.962800
4,3.709200
5,3.941200
6,3.883000
7,3.940700
8,3.896000
9,3.827100
10,3.866100


In [14]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
{}
</مسئله>
"""
message = sys_prompt.format("3 به علاوه 5 چند می شود؟")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
3 به علاوه 5 چند می شود؟
</مسئله>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

در ابتدا فکر کردم ۸ می‌شود درست باشد، اما شک دارم.
اشتباه اولیه را کنار می‌گذارم؛ پاسخ درست ۸ می‌شود است.
<|eot_id|>


In [16]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
{}
</مسئله>
"""
message = sys_prompt.format("10 به علاوه 20 چند می شود؟")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

In [17]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
10 به علاوه 20 چند می شود؟
</مسئله>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

اول به ۳۰ می‌شود رسید، اما ممکن است مسیرم غلط باشد. شاید دارم عجله می‌کنم.
با دقت بیشتر، نتیجه نهایی ۳۰ می‌شود به دست می‌آید.
<|eot_id|>


In [18]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
{}
</مسئله>
"""
message = sys_prompt.format("کلمه 'جارو' چند حرف 'ر' دارد؟")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

In [19]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
کلمه 'جارو' چند حرف 'ر' دارد؟
</مسئله>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

به نظر می‌رسد ۱ حرف «ر» دارد باشد، اما باید دوباره بررسی شود.
تحلیل دوباره نشان می‌دهد که پاسخ ۱ حرف «ر» دارد درست است.
<|eot_id|>


In [33]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
{}
</مسئله>
"""
message = sys_prompt.format("معنی جمله 'hello how are you' به فارسی چیست؟")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 0.1, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

In [34]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

تو یک دستیار متفکر هستی که با استدلال دقیق و تکرار شوند، تفکر جریان سیال ذهن را تقلید می کنی. رویکرد تو بر کاوش، شک به خود و اصلاح مداوم قبل از رسیدن به پاسخ تاکید دارد.
<مسئله>
معنی جمله 'hello how are you' به فارسی چیست؟
</مسئله>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

در ابتدا فکر کردم پاسخ «کلمه «سلام how are you» به فارسی «کلمه «سلام، چگونه می‌رسم» است» باشد، اما شک دارم.
با بررسی دقیق‌تر مشخص می‌شود پاسخ درست «کلمه «سلام، چگونه می‌رسم» است است.
<|eot_id|>
